In [1]:

from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm
import os
from glob import glob
import pandas as pd
import json
from skimage.draw import polygon2mask
import shutil

In [3]:
class_list=['NT_epithelial',
 'NT_Muscle',
 'NT_stroma',
 'NT_immune',
 'NT_intestinal_metaplasia']
json_list=glob('../../data/origin_json/STNT/유형*/*.json')
image_path="../../data/origin_json/STNT/**/"
save_img_path='../../data/area_segmentation/STNT/image/'
save_mask_path='../../data/area_segmentation/STNT/mask/'

In [4]:

for j in tqdm(range(len(json_list))):
    with open(json_list[j], 'r') as f:
        json_data = json.load(f)
    anno=json_data['content']['file']['objects']
    file_name=json_data['content']['file']['filename']
    # for i in range(len(anno)):
    #     class_list.append(anno[i]['label_nm'])
    # class_list=list(set(class_list))
    ii1=glob(image_path+file_name)[0]
    shutil.copyfile(ii1, save_img_path+file_name)
    mask=np.zeros((1024,1024,len(class_list)+1))
    for k in range(len(anno)):
        coordinate=np.array(anno[k]['coordinate'])
        coordinate[:,[0, 1]]=coordinate[:,[1, 0]]
        class_nm=anno[k]['label_nm']
        if class_nm=='NT_gastritis':
            class_nm='NT_immune'
        mask_temp=polygon2mask((1024,1024), coordinate)
        mask[...,class_list.index(class_nm)+1]+=mask_temp
    mask[...,0]=np.where(mask[...,1:].sum(axis=2)>=1,0,1)
    mask=np.where(mask>=1,255,0)
    reversed_index = np.argmax(mask[:,:,::-1], axis=2)
    final_index = len(class_list)-reversed_index
    for i in range(len(class_list)):
        if not os.path.exists(save_mask_path+class_list[i]):
            os.makedirs(save_mask_path+class_list[i])
        cv2.imwrite(save_mask_path+class_list[i]+'/'+file_name,np.where(final_index==i+1,255,0))
    np.save(save_mask_path+'npy/'+file_name.replace('.jpeg','.npy'),final_index)

100%|██████████| 308/308 [03:45<00:00,  1.37it/s]
